In [7]:
import numpy as np
import matplotlib.pyplot as plt

from fixedpoint import FixedPoint

In [8]:
def generateurMandelbrot(z_re, z_im, c_re, c_im):
    z2_re = z_re**2
    z2_im = z_im**2
    r2 = z2_re + z_im

    z_re1 = z2_re - z2_im + c_re
    z_im1 = 2 * z_re*z_im + c_im

    # entre chaque calcul tronc n apres la virgule n > m

    return z_re1, z_im1, r2

def iterateur(z_re, z_im, c_re, c_im, limR=4, limIter=100):
    iteration = 0
    r2 = 0
    while((r2 < limR) and (iteration < limIter)):
        (z_re, z_im, r2) = generateurMandelbrot(z_re, z_im, c_re, c_im)
        iteration += 1
    return iteration

In [9]:
m = 4
n = 14

def createFP(v):
    return FixedPoint(v, m=m, n=n, signed=True, str_base=2)

def resize(x : FixedPoint):
    #x_str = str(x)
    x_str = f"{x.bits:0{x.m+x.n}b}"
    new = f"0b{x_str[x.m-m:x.m+n]}"
    #print(f"[{start}:{end}] -> {new}")
    return FixedPoint(new, m=m, n=n, signed=True, str_base=2)

def FP_generateurMandelbrot(z_re : FixedPoint, z_im : FixedPoint, c_re : FixedPoint, c_im : FixedPoint):  
    z2_re = z_re*z_re
    z2_re = resize(z2_re)

    z2_im = z_im*z_im
    z2_im = resize(z2_im)

    r2 = z2_re + z_im
    r2 = resize(r2)

    z_re1_int = z2_re - z2_im   
    z_re1_int = resize(z_re1_int)
    z_re1_int = z_re1_int + c_re
    z_re1 = resize(z_re1_int)

    z_im1_int = 2 * z_re
    z_im1_int = resize(z_im1_int)
    z_im1_int = z_im1_int*z_im
    z_im1_int = resize(z_im1_int)
    z_im1_int = z_im1_int + c_im
    z_im1 = resize(z_im1_int)
    # entre chaque calcul tronc n apres la virgule n > m

    return z_re1, z_im1, r2

def FP_iterateur(z_re, z_im, c_re, c_im, limR=4, limIter=100, save=False):
    iteration = 0
    r2 = 0
    if save:
        tabZ_re = []
        tabZ_im = []
    # prepare for FixedPoint  
    z_re = createFP(z_re)
    z_im = createFP(z_im)
    c_re = createFP(c_re)
    c_im = createFP(c_im)
    limFP = createFP(limR)
    while((r2 < limFP) and (iteration < limIter)):
        (z_re, z_im, r2) = FP_generateurMandelbrot(z_re, z_im, c_re, c_im)
        if save:
            tabZ_re.append(z_re)
            tabZ_im.append(z_im)
        #print(float(r2))
        iteration += 1
    if save:
        return iteration, tabZ_re, tabZ_im
    return iteration

In [10]:
xlim = 612
ylim = 300
limI = 50
figure_float = np.zeros((xlim,ylim))
figure_fixed = np.zeros((xlim,ylim))
error = np.zeros((xlim,ylim))
xvec = np.linspace(-2,1,xlim)
yvec = np.linspace(-1,1,ylim)

In [11]:
for ix,x in enumerate(xvec):
    for iy,y in enumerate(yvec):
        figure_float[ix,iy] = iterateur(0,0,x,y,limIter=limI)

In [12]:
for ix,x in enumerate(xvec):
    for iy,y in enumerate(yvec):
        figure_fixed[ix,iy] = FP_iterateur(0,0,x,y,limIter=limI)
            

In [ ]:
for ix,x in enumerate(xvec):
    for iy,y in enumerate(yvec):
        error = abs(figure_float[ix,iy] - figure_fixed[ix,iy])

In [ ]:
plt.figure()
plt.imshow(figure_float.T)
plt.title("Virgule flottante")
plt.show()

plt.figure()
plt.imshow(figure_fixed.T)
plt.title("Virgule fixe")
plt.show()

plt.figure()
plt.imshow(error.T)
plt.title("Erreur entre vrigule flottante et fixe")
plt.show()